In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/netflow/CIDDS-001/traffic/mixed.csv', index_col='DATE')
df.head()

,DURATION,PROTOCOL,SRC_IP_ADDR,SRC_PORT,DST_IP_ADDR,DST_PORT,PACKETS,BYTES,FLAGS,TOS,CLASS,ATTACK_TYPE,ATTACK_ID,ATTACK_DESCRIPTION,TYPE
DATE,,,,,,,,,,,,,,,
2017-03-14 17:43:26.135,81504.787,TCP,EXT_SERVER,8082,OPENSTACK_NET,56979,8639,9318.4,.AP...,0,normal,benign,0.0,benign,external
2017-03-14 17:43:26.135,81504.787,TCP,OPENSTACK_NET,56979,EXT_SERVER,8082,12024,10547.2,.AP...,0,normal,benign,0.0,benign,external
2017-03-14 17:43:26.135,81504.787,TCP,OPENSTACK_NET,56979,EXT_SERVER,8082,12024,10547.2,.AP...,0,normal,benign,0.0,benign,internal
2017-03-14 17:43:26.135,81504.787,TCP,EXT_SERVER,8082,OPENSTACK_NET,56979,8639,9318.4,.AP...,0,normal,benign,0.0,benign,internal
2017-03-14 17:43:39.011,183418.493,TCP,OPENSTACK_NET,60802,EXT_SERVER,8082,20751,5939.2,.AP...,0,normal,benign,0.0,benign,internal


In [3]:
df.drop(['FLAGS', 'CLASS', 'ATTACK_ID', 'ATTACK_DESCRIPTION'], axis=1, inplace=True)

In [4]:
df = df.sample(frac=1).iloc[:1000]
df.head()

,DURATION,PROTOCOL,SRC_IP_ADDR,SRC_PORT,DST_IP_ADDR,DST_PORT,PACKETS,BYTES,TOS,ATTACK_TYPE,TYPE
DATE,,,,,,,,,,,
2017-03-24 08:00:31.555,0.000,TCP,11250_39,80,192.168.220.15,56170,2,132.0,32,benign,internal
2017-03-27 10:51:19.112,0.580,TCP,11067_110,443,192.168.220.14,43016,3,229.0,32,benign,internal
2017-03-23 17:33:57.275,0.373,TCP,192.168.220.7,58101,10796_87,443,53,5885.0,0,benign,internal
2017-03-22 16:37:37.799,0.000,TCP,10321_204,443,192.168.220.15,38800,1,66.0,0,benign,internal
2017-03-24 15:19:59.626,0.000,TCP,192.168.220.13,35092,11931_215,443,1,66.0,0,benign,internal


In [5]:
df['ATTACK_TYPE'].value_counts(normalize=True) * 100

benign        85.4
dos           12.8
portScan       1.6
bruteForce     0.2
Name: ATTACK_TYPE, dtype: float64

In [6]:
df['TYPE'].value_counts(normalize=True) * 100

internal    92.4
external     7.6
Name: TYPE, dtype: float64

In [7]:
df['SRC'] = df['SRC_IP_ADDR'].str.cat(df['SRC_PORT'].astype(str), sep=':')
df['DST'] = df['DST_IP_ADDR'].str.cat(df['DST_PORT'].astype(str), sep=':')

df.drop(['SRC_IP_ADDR', 'SRC_PORT', 'DST_IP_ADDR', 'DST_PORT'], axis=1, inplace=True)

In [8]:
df.head()

,DURATION,PROTOCOL,PACKETS,BYTES,TOS,ATTACK_TYPE,TYPE,SRC,DST
DATE,,,,,,,,,
2017-03-24 08:00:31.555,0.000,TCP,2,132.0,32,benign,internal,11250_39:80,192.168.220.15:56170
2017-03-27 10:51:19.112,0.580,TCP,3,229.0,32,benign,internal,11067_110:443,192.168.220.14:43016
2017-03-23 17:33:57.275,0.373,TCP,53,5885.0,0,benign,internal,192.168.220.7:58101,10796_87:443
2017-03-22 16:37:37.799,0.000,TCP,1,66.0,0,benign,internal,10321_204:443,192.168.220.15:38800
2017-03-24 15:19:59.626,0.000,TCP,1,66.0,0,benign,internal,192.168.220.13:35092,11931_215:443


In [9]:
from sklearn.preprocessing import LabelEncoder

def label_encode(series_to_encode):
    lb_e = LabelEncoder()
    return lb_e, lb_e.fit_transform(series_to_encode)

In [10]:
FEATURES_TO_LABELENCODE = ['PROTOCOL', 'TYPE', 'TOS', 'ATTACK_TYPE', 'SRC', 'DST']

In [11]:
encoders = {}

for f in FEATURES_TO_LABELENCODE:
    print("Processing feature {}".format(f))
    encoders[f], df[f] = label_encode(df[f])

Processing feature PROTOCOL
Processing feature TYPE
Processing feature TOS
Processing feature ATTACK_TYPE
Processing feature SRC
Processing feature DST


In [12]:
df.head()

,DURATION,PROTOCOL,PACKETS,BYTES,TOS,ATTACK_TYPE,TYPE,SRC,DST
DATE,,,,,,,,,
2017-03-24 08:00:31.555,0.000,2,2,132.0,1,0,1,194,550
2017-03-27 10:51:19.112,0.580,2,3,229.0,1,0,1,190,409
2017-03-23 17:33:57.275,0.373,2,53,5885.0,0,0,1,675,184
2017-03-22 16:37:37.799,0.000,2,1,66.0,0,0,1,125,459
2017-03-24 15:19:59.626,0.000,2,1,66.0,0,0,1,388,217


In [13]:
df['ATTACK_TYPE'].value_counts(normalize=True) * 100

0    85.4
2    12.8
3     1.6
1     0.2
Name: ATTACK_TYPE, dtype: float64

In [14]:
df['TYPE'].value_counts(normalize=True) * 100

1    92.4
0     7.6
Name: TYPE, dtype: float64

In [16]:
import networkx as nx

In [17]:
import plotly.graph_objects as go

In [49]:
G = nx.MultiGraph()

In [50]:
nodes_df = pd.concat([
    df[['SRC', 'TYPE']].rename(columns={'SRC': 'ID'}).reset_index(drop=True), 
    df[['DST', 'TYPE']].rename(columns={'DST': 'ID'}).reset_index(drop=True)
]).drop_duplicates()

In [51]:
nodes_df.head()

,ID,TYPE
0,194,1
1,190,1
2,675,1
3,125,1
4,388,1


In [52]:
nodes_df['TYPE'].unique()

array([1, 0])

In [56]:
nodes_df['TYPE'].value_counts(normalize=True) * 100

1    89.878049
0    10.121951
Name: TYPE, dtype: float64

In [57]:
import numpy as np

nodes_df['COLOR'] = np.where(nodes_df['TYPE'] == 0, "#107be6", "#000000")
nodes_df.drop('TYPE', axis=1, inplace=True)

In [58]:
nodes_df.head()

,ID,COLOR
0,194,#000000
1,190,#000000
2,675,#000000
3,125,#000000
4,388,#000000


In [59]:
nodes_list = nodes_df.to_dict('records')

In [60]:
G.add_nodes_from([(node['ID'], {"color": node["COLOR"]}) for node in nodes_list])

In [62]:
edges_df = df[['SRC', 'DST', 'DURATION', 'PROTOCOL', 'PACKETS', 'BYTES', 'TOS', 'ATTACK_TYPE']]
edges_df['SPEED'] = edges_df['BYTES'] / edges_df['DURATION']
edges_df['SPEED'] = edges_df['SPEED'].replace([np.inf, -np.inf], 0.01)


edges_df.reset_index(inplace=True, drop=True)

In [63]:
edges_df.head()

,SRC,DST,DURATION,PROTOCOL,PACKETS,BYTES,TOS,ATTACK_TYPE,SPEED
0,194,550,0.000,2,2,132.0,1,0,0.010000
1,190,409,0.580,2,3,229.0,1,0,394.827586
2,675,184,0.373,2,53,5885.0,0,0,15777.479893
3,125,459,0.000,2,1,66.0,0,0,0.010000
4,388,217,0.000,2,1,66.0,0,0,0.010000


In [64]:
for e in edges_df.to_dict('records'):
#     for w in ['DURATION', 'PACKETS', 'BYTES', 'SPEED']:
#         G.add_weighted_edges_from([(e['SRC'], e['DST'], e[w])], color={0: 'blue'}.get(e['ATTACK_TYPE'], "red"))

    G.add_weighted_edges_from(
        [(e['SRC'], e['DST'], e['SPEED'])], 
        color={
            0: '#299e50',
            1: "#ff0000",
            2: "#ff0000"
        }.get(e['ATTACK_TYPE'], "#000000")
    )

In [65]:
N = G.number_of_nodes()

In [66]:
from plotly.graph_objs import *

In [67]:
import plotly.express as px

In [68]:
pos = nx.random_layout(G)

In [69]:
color_by_nodes = {x[0]: x[1]['color'] for x in G.nodes.data()}
nodes_colors = [color_by_nodes[x] for x in pos]

In [ ]:
# Xed, Yed, edges_colors, weights = [], [], [], []

# for edge in G.edges.data():
    
#     Xed += (pos[edge[0]][0], pos[edge[1]][0])
#     Yed += (pos[edge[0]][1], pos[edge[1]][1])
    
#     edges_colors += ({0: 'blue'}.get(edge[2]['color'], 'red'), {0: 'blue'}.get(edge[2]['color'], 'red'))
#     weights += [edge[2]['weight']]

In [ ]:
# trace_ed = Scatter(x=Xed, 
#                    y=Yed, 
#                    mode='lines', 
#                    line=dict(width=2, color=colors[k])) for k, e in enumerate(G.edges()), 
#                    hoverinfo='all')

# trace_ed = px.scatter(x=Xed, y=Yed, color=colors, size=weights)


# trace_ed = [Scatter(x=[pos[e[0]][0], pos[e[1]][0]],
#                     y=[pos[e[0]][1], pos[e[1]][1]],
#                     mode='lines',
#                     line=dict(width=2, color=colors[k])) for k, e in enumerate(G.edges())
#            ]

In [ ]:
# trace_v = Scatter(x=Xv, 
#                   y=Yv, 
#                   mode='markers', 
#                   name='net',
#                   marker=dict(symbol='circle-dot',
#                               size=6,
#                               color='#6959CD',
#                               line=dict(color='rgb(50,50,50)', width=0.5)
#                              ),
#                   hoverinfo='text'
#                  )

In [ ]:
# nodes = dict(
#            type='scatter',
#            x=Xv,
#            y=Yv,
#            mode='markers',
#            name='net',
#            marker=dict(symbol='circle-dot',
#                        size=6,
#                        color='#6959CD',
#                        line=dict(color='rgb(50,50,50)', width=0.5)
#                   )
#           )

# edges_list=[dict(
#              type='scatter',
#              x=Xed,
#              y=Yed,
#              mode='lines',
#              line=dict(width=2, color=colors[k])) for k, e in enumerate(G.edges())
#            ]

In [71]:
width=1100
height=900

In [72]:
axis=dict(showline=False, # hide axis line, grid, ticklabels and  title
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title='Netflow multigraph data'
          )

In [73]:
_layout=Layout(
    font= dict(size=12),
    showlegend=False,
    autosize=False,
    width=width,
    height=height,
    xaxis=layout.XAxis(axis),
    yaxis=layout.YAxis(axis),
    margin=layout.Margin(
        l=40,
        r=40,
        b=85,
        t=100,
    ),
    hovermode='closest',
    annotations=[
           dict(
               showarrow=False,
                text='#PFA (WIP)',
                xref='paper',
                yref='paper',
                x=0,
                y=-0.1,
                xanchor='left',
                yanchor='bottom',
                font=dict(
                    size=14
                )
           )
    ]
)

In [ ]:
# fig=Figure(data=[trace_ed, trace_v], layout=_layout)
# fig['layout']['annotations'][0]['text']= "Netflow graphs"

# fig.show()

In [75]:
Xv, Yv = [pos[k][0] for k in range(N)], [pos[k][1] for k in range(N)]
nodes_scatter = go.Scattergl(x=Xv, y=Yv, mode='markers', marker=dict(size=8, color=nodes_colors, line_width=1))

In [76]:
edges_list = [
                Scatter(x=[pos[e[0]][0], pos[e[1]][0]], 
                        y=[pos[e[0]][1], pos[e[1]][1]],
                        mode='lines',
                        line=dict(width=0.5, color=e[2]['color'])
                       ) for k, e in enumerate(G.edges.data())
             ]

In [77]:
edges_list.append(nodes_scatter)

In [78]:
import plotly.graph_objects as go

N = 100000
fig = go.Figure(
    data=edges_list,
    layout=_layout
)

fig.show()